### High-level implementation of Shor's algorithm (builds operators element-by-element, not gate-by-gate). Due to memory limitations, only small non-trivial cases (e.g. 15) can be factorised.

---

In [1]:
from MiQE import *
from fractions import Fraction

In [2]:
def mod_mult_gate(b, N):

    if np.gcd(b, N) > 1:
        raise ValueError(f'gcd({b}, {N}) > 1.')
    else:
        n = int(np.ceil(np.log2(N-1)))
        U = np.zeros((2**n, 2**n), dtype=complex)

        for x in range(N):
            U[b*x % N, x] = 1
        for x in range(N, 2**n):
            U[x, x] = 1

        return U

In [3]:
def controlled_mod_mult_gate(circuit, b, N, control, target):
    
    proj_0 = np.array([[1, 0], [0, 0]], dtype=complex)
    proj_1 = np.array([[0, 0], [0, 1]], dtype=complex)

    G = mod_mult_gate(b, N)
    t = len(target)
    num_qubits = circuit.n

    inactive_path = []
    active_path = []

    i = 0
    while i < num_qubits:
        if i == control:
            inactive_path.append(proj_0)
            active_path.append(proj_1)
            i += 1
        elif i == target[0]:
            inactive_path += [I] * t
            active_path.append(G)
            i += t
        else:
            inactive_path.append(I)
            active_path.append(I)
            i += 1

    expanded_gate = reduce(np.kron, inactive_path) + reduce(np.kron, active_path) 
    circuit.state = expanded_gate @ circuit.state

In [4]:
def inverse_QFT(circuit, m):
    
    N = 2**m
    omega = np.exp(2j * np.pi / N)
    
    j = np.arange(N).reshape((N, 1))
    k = np.arange(N).reshape((1, N))
    QFT = omega ** (j * k)

    QFT /= np.sqrt(N)
    QFT_inv = np.linalg.inv(QFT)

    path = []
    path.append(QFT_inv)
    # for i in range(circuit.n - m):
    #     path.append(I)
    path += [I] * (circuit.n - m)

    expanded_gate = reduce(np.kron, path)
    circuit.state = expanded_gate @ circuit.state

In [5]:
def order_finding_circuit(a, N):

    if np.gcd(a, N) > 1:
        raise ValueError(f'gcd({a}, {N}) > 1.')
    else:
        n = int(np.ceil(np.log2(N-1)))
        m = 2*n

        control = [qubit for qubit in range(m)]
        target = [qubit + m for qubit in range(n)]
        circuit = QuantumCircuit(m + n)

        circuit.gate(X, m)

        for k, qubit in enumerate(control):
            circuit.gate(H, k)
            b = pow(a, 2**k, N)
            controlled_mod_mult_gate(circuit, b, N, control=qubit, target=target)

        inverse_QFT(circuit, m)

        measurement = circuit.measure_all()[:-n]

        return measurement

In [6]:
def find_order(a, N, max_attempts=30):
    
    if np.gcd(a, N) > 1:
        raise ValueError(f'gcd({a}, {N}) > 1.')
    else:
        n = int(np.ceil(np.log2(N-1)))
        m = 2*n

        for attempt in range(max_attempts): 
            measurement = order_finding_circuit(a, N)
            
            y = int(measurement[::-1], 2)
            r = Fraction(y / 2**m).limit_denominator(N).denominator
            
            print(f'Attempt {attempt+1}: measurement = {y}, order approx = {r}')
            
            if pow(a, r, N) == 1:
                return r
    
        raise RuntimeError(f'Failed to find a valid order of {a} modulo {N} after {max_attempts} attempts.')

In [7]:
N = 15

factor_found = False

if N % 2 == 0:
    print('Even number')
    d = 2
    factor_found = True
else:
    for k in range(2, round(np.log2(N)) + 1):
        d = int(round(N**(1/k)))
        if d**k == N:
            factor_found = True
            print('Number is a power')
            break

while not factor_found:
    a = np.random.randint(2, N-1)
    d = np.gcd(a, N)
    if d > 1:
        factor_found = True
        print(f'Lucky guess of {a} modulo {N}')
    else:
        r = find_order(a, N)
        print(f'\nThe order of {a} modulo {N} is {r}')
        if r % 2 == 0:
            x = pow(a, r//2, N) - 1
            d = np.gcd(x, N)
            if d > 1 and d != N:
                factor_found = True

print(f'Factor found: {d}')
print(f'\t\033[1mSolution\033[0m: {N} = {d} * {int(N/d)}')

Attempt 1: measurement = 0, order approx = 1
Attempt 2: measurement = 128, order approx = 2
Attempt 3: measurement = 64, order approx = 4

The order of 7 modulo 15 is 4
Factor found: 3
	Solution: 15 = 3 * 5
